<h1>KNN - K Nearest Neighbor</h1>

In [8]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

<h2>1. Dataprep</h2>

In [9]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [10]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [11]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [12]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [13]:
cl = "classif.kknn"

List all parameters that can be used in this classifier. The value must be set in par.vals parameter.

In [14]:
getParamSet(cl)

             Type len     Def                                   Constr Req
k         integer   -       7                                 1 to Inf   -
distance  numeric   -       2                                 0 to Inf   -
kernel   discrete   - optimal rectangular,triangular,epanechnikov,b...   -
scale     logical   -    TRUE                                        -   -
         Tunable Trafo
k           TRUE     -
distance    TRUE     -
kernel      TRUE     -
scale       TRUE     -

In [15]:
learner = makeLearner(cl = "classif.kknn"
                     , predict.type = "prob"
                     , par.vals = list(k = 10)
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [16]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
)

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [17]:
resample = makeResampleDesc(
    method = "CV",
    iters = 10,
    predict = 'both',
    stratify = FALSE
)

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [18]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               #,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [19]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE)

Resampling: cross-validation
Measures:             mmce.test   acc.test    f1.test     ppv.test    tpr.test    auc.test    gini.test   
[Resample] iter 1:    0.2000000   0.8000000   0.5238095   0.7333333   0.4074074   0.7283105   0.4566210   
[Resample] iter 2:    0.2900000   0.7100000   0.4313725   0.6111111   0.3333333   0.6720941   0.3441882   
[Resample] iter 3:    0.2700000   0.7300000   0.4905660   0.6500000   0.3939394   0.7365445   0.4730891   
[Resample] iter 4:    0.3100000   0.6900000   0.4150943   0.5238095   0.3437500   0.7192096   0.4384191   
[Resample] iter 5:    0.2600000   0.7400000   0.4800000   0.6000000   0.4000000   0.7788095   0.5576190   
[Resample] iter 6:    0.2600000   0.7400000   0.4583333   0.6470588   0.3548387   0.7428705   0.4857410   
[Resample] iter 7:    0.2400000   0.7600000   0.5862069   0.6800000   0.5151515   0.7720488   0.5440977   
[Resample] iter 8:    0.2000000   0.8000000   0.5238095   0.7333333   0.4074074   0.7841197   0.5682395   
[Resampl

<h2>3. Result Analysis</h2>

Train Measures

In [20]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.1211111,0.8788889,0.7655914,0.9270833,0.6520147,0.9716073,0.9432147
2,0.1166667,0.8833333,0.7692308,0.9308511,0.6554307,0.9723213,0.9446427
3,0.1188889,0.8811111,0.7616927,0.9395604,0.6404494,0.9744869,0.9489737
4,0.1211111,0.8788889,0.7614880,0.9206349,0.6492537,0.9730836,0.9461671
5,0.1266667,0.8733333,0.7500000,0.9193548,0.6333333,0.9701646,0.9403292
6,0.1222222,0.8777778,0.7598253,0.9206349,0.6468401,0.9697418,0.9394836
7,0.1233333,0.8766667,0.7560440,0.9148936,0.6441948,0.9699724,0.9399447
8,0.1266667,0.8733333,0.7553648,0.9119171,0.6446886,0.9696794,0.9393589
9,0.1255556,0.8744444,0.7590618,0.9175258,0.6472727,0.9744465,0.9488931
10,0.1222222,0.8777778,0.7619048,0.9214660,0.6494465,0.9732604,0.9465209


Test Measures

In [21]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.20,0.80,0.5238095,0.7333333,0.4074074,0.7283105,0.4566210
2,0.29,0.71,0.4313725,0.6111111,0.3333333,0.6720941,0.3441882
3,0.27,0.73,0.4905660,0.6500000,0.3939394,0.7365445,0.4730891
4,0.31,0.69,0.4150943,0.5238095,0.3437500,0.7192096,0.4384191
5,0.26,0.74,0.4800000,0.6000000,0.4000000,0.7788095,0.5576190
6,0.26,0.74,0.4583333,0.6470588,0.3548387,0.7428705,0.4857410
7,0.24,0.76,0.5862069,0.6800000,0.5151515,0.7720488,0.5440977
8,0.20,0.80,0.5238095,0.7333333,0.4074074,0.7841197,0.5682395
9,0.23,0.77,0.4102564,0.5714286,0.3200000,0.7320000,0.4640000
10,0.24,0.76,0.4782609,0.6470588,0.3793103,0.7661486,0.5322972


Train Aggregated Result

In [22]:
apply(r$measures.train,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.1224444 0.8775556 0.7600203 0.9223922 0.6462925 0.9718764 0.9437529

Test Aggregated Result

In [23]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2500000 0.7500000 0.4797709 0.6397134 0.3855138 0.7432156 0.4864312

Run Time in seconds

In [24]:
r$runtime

[1] 2.471726

<h2>4. Prediction for new data</h2>

Read the data to predict

In [25]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [26]:
best.model = which.max(r$measures.test$acc)

In [27]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [28]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.40
  truth    prob.1     prob.2 response
1     1 0.9822112 0.01778882        1
2     2 0.2957408 0.70425921        2
3     1 1.0000000 0.00000000        1
4     1 0.6284030 0.37159702        1
5     2 0.2002128 0.79978718        2
6     1 0.7618780 0.23812202        1
... (#rows: 1000, #cols: 4)

Cast result to data.frame to access the prediction

In [29]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.9822112,0.01778882,1
2,0.2957408,0.70425921,2
1,1.0000000,0.00000000,1
1,0.6284030,0.37159702,1
2,0.2002128,0.79978718,2
1,0.7618780,0.23812202,1
